In [ ]:
import matplotlib.pyplot as plt
import meow as mw
import numpy as np
import tidy3d as td
from matplotlib import colors
from scipy.ndimage import convolve
from tidy3d.plugins.mode.mode_solver import compute_modes as _compute_modes

mw.cache.disable_cache();

## Settings

In [ ]:
T = 25.0
height = 0.22
length = 10
num_modes = 5
width_in = 0.45
width_out = 2.0
wl = 1.5
cell_length = 2.0

## Meow Structures

In [ ]:
oxide = mw.Structure(
    material=mw.silicon_oxide,
    geometry=mw.Prism(
        poly=np.array(
            [
                (-2, -3.1),
                (length + 2, -3.1),
                (length + 2, 3.1),
                (-2, 3.1),
            ]
        ),
        h_min=-3,
        h_max=0,
        axis="y",
    ),
)

poly = np.array(
    [
        (-2, -width_in / 2),
        (0, -width_in / 2),
        (length, -width_out / 2),
        (length + 2, -width_out / 2),
        (length + 2, width_out / 2),
        (length, width_out / 2),
        (0, width_in / 2),
        (-2, width_in / 2),
    ]
)

core = mw.Structure(
    material=mw.silicon,
    geometry=mw.Prism(
        poly=poly,
        h_min=0,
        h_max=0.22,
        axis="y",
    ),
)

structures = [oxide, core]

cs1 = mw.CrossSection(
    cell=mw.Cell(
        structures=structures,
        mesh=mw.Mesh2d(
            x=np.linspace(-2, 2, round(4 / (wl / 20))),
            y=np.linspace(-2, 2, round(4 / (wl / 20))),
        ),
        z_min=0,
        z_max=0,
    ),
    env=mw.Environment(wl=wl, T=T),
    ez_interfaces=False,
)

mw.visualize(cs1)

In [ ]:
modes1 = mw.compute_modes(cs=cs1, num_modes=5)
mw.visualize(modes1, fields=["Ey", "Hy"], plot_width=3)

## Current

In [ ]:
cmap1 = colors.LinearSegmentedColormap.from_list(
    name="cmap1", colors=["#ffffff", "#ff0000"]
)
cmap2 = colors.LinearSegmentedColormap.from_list(
    name="cmap2", colors=["#ffffff", "#00ff00"]
)
cmap3 = colors.LinearSegmentedColormap.from_list(
    name="cmap3", colors=["#ffffff", "#0000ff"]
)

In [ ]:
plt.pcolormesh(
    cs1.cell.mesh.Xx + 0.5,
    cs1.cell.mesh.Yx + 0.5,
    cs1.nx,
    cmap=cmap1,
    vmin=1.0,
    alpha=0.3,
)
plt.pcolormesh(
    cs1.cell.mesh.Xy + 0.5,
    cs1.cell.mesh.Yy + 0.5,
    cs1.ny,
    cmap=cmap2,
    vmin=1.0,
    alpha=0.3,
)
plt.pcolormesh(
    cs1.cell.mesh.Xz + 0.5,
    cs1.cell.mesh.Yz + 0.5,
    cs1.nz,
    cmap=cmap3,
    vmin=1.0,
    alpha=0.3,
)
plt.show()

## Desired

In [ ]:
from meow.cross_section import (
    _get_boundary_mask_vertical,
    _get_boundary_mask_horizontal,
    _fill_corner_left_mask,
    _fill_corner_right_mask,
)

x_full = cs1.cell.mesh.x_full.copy()
y_full = cs1.cell.mesh.y_full.copy()
X_full = cs1.cell.mesh.X_full.copy()
Y_full = cs1.cell.mesh.Y_full.copy()
n_full = cs1.n_full.copy()

mask_ez_horizontal = np.zeros_like(n_full, dtype=bool)
mask_ez_horizontal[:, ::2] = True
mask_ez_vertical = np.zeros_like(n_full, dtype=bool)
mask_ez_vertical[::2, :] = True
mask_boundaries_vertical = _get_boundary_mask_vertical(n_full)
mask_boundaries_vertical = mask_boundaries_vertical & (~mask_ez_vertical)
mask_boundaries_horizontal = _get_boundary_mask_horizontal(n_full)
mask_boundaries_horizontal = mask_boundaries_horizontal & (~mask_ez_horizontal)
mask_temp = mask_boundaries_vertical  # | mask_boundaries_horizontal
mask_corner_left = _fill_corner_left_mask(mask_temp)
mask_corner_right = _fill_corner_right_mask(mask_temp)
mask_to_remove = mask_temp | mask_corner_left | mask_corner_right

mask_to_keep = (n_full > 1) & (~mask_to_remove)

mask_ez = np.zeros_like(n_full, dtype=bool)
mask_ez[::2, ::2] = True
final_mask_to_keep = mask_to_keep & mask_ez

mask_ex = np.zeros_like(n_full, dtype=bool)
mask_ex[1::2, ::2] = True
final_mask_to_keep |= mask_to_keep & mask_ex

mask_ey = np.zeros_like(n_full, dtype=bool)
mask_ey[::2, 1::2] = True
final_mask_to_keep |= mask_to_keep & mask_ey

mask_hz = np.zeros_like(n_full, dtype=bool)
mask_hz[1::2, 1::2] = True
final_mask_to_keep |= mask_to_keep & mask_hz

n_full[~final_mask_to_keep] = 1.0

plt.pcolormesh(X_full, Y_full, n_full, cmap="Blues")
plt.pcolormesh(X_full, Y_full, mask_to_remove, cmap="Reds", alpha=0.5)

dx = cs1.cell.mesh.x[1:] - cs1.cell.mesh.x[:-1]
dy = cs1.cell.mesh.y[1:] - cs1.cell.mesh.y[:-1]
x_ticks = np.sort(np.unique(X_full.ravel()))[::2]
y_ticks = np.sort(np.unique(Y_full.ravel()))[::2]
plt.xticks(x_ticks - 0.25 * dx, [f"" for x in x_ticks - 0.25 * dx])
plt.yticks(y_ticks - 0.25 * dy, [f"" for y in y_ticks - 0.25 * dy])
plt.xticks(x_ticks + 0.25 * dx, [f"" for x in x_ticks + 0.25 * dx], minor=True)
plt.yticks(y_ticks + 0.25 * dy, [f"" for y in y_ticks + 0.25 * dy], minor=True)
plt.grid(True, which="major", ls="-")
plt.grid(True, which="minor", ls=":")
plt.xlim(-0.5, 0.5)
plt.ylim(-0.11, 0.33)
plt.show()

You can see that removing some mesh points horizontally will place the silicon on slightly higher bed of silicon oxide. Even though this is not ideal, I think it's still better than not doing this extra meshing cleanup step. That being said... it would be great if we could fix this too in the future.